Name: Karel Mackenzie Chandra
Student ID:30373867

From Part A

In [1]:
import pymongo
from pymongo import MongoClient
from pprint import pprint

# Method 1: connect on the default host and port
client = MongoClient () # method 1: connect on the default host and port

In [2]:
db = client.fit3182_assignment_db # assume that we use the database fit3182_db that we created in the previous tutorials.
#db = client['fit3182_db'] # another way of getting a database

In [3]:
# Data Model from part A
Embedded = db.EMBEDDED
# Loop to check if data exists in the collection
emb = Embedded.find({})
for document in emb: 
    pprint(document)

{'GHI_w/m2': 154,
 'Precipitation': 0.0,
 'Type': 'I',
 '_id': ObjectId('628ac5d0f8b13d75e14120d4'),
 'air_temperature_celcius': 19,
 'date': '2020-12-31',
 'hotspots': [],
 'max_wind_speed': 11.1,
 'relative_humidity': 56.8,
 'station': 948700,
 'windspeed_knots': 7.9}
{'GHI_w/m2': 128,
 'Precipitation': 0.02,
 'Type': 'G',
 '_id': ObjectId('628ac5d0f8b13d75e14120d5'),
 'air_temperature_celcius': 15,
 'date': '2021-01-02',
 'hotspots': [],
 'max_wind_speed': 13.0,
 'relative_humidity': 50.7,
 'station': 948700,
 'windspeed_knots': 9.2}
{'GHI_w/m2': 133,
 'Precipitation': 0.0,
 'Type': 'G',
 '_id': ObjectId('628ac5d0f8b13d75e14120d6'),
 'air_temperature_celcius': 16,
 'date': '2021-01-03',
 'hotspots': [],
 'max_wind_speed': 15.0,
 'relative_humidity': 53.6,
 'station': 948700,
 'windspeed_knots': 8.1}
{'GHI_w/m2': 186,
 'Precipitation': 0.0,
 'Type': 'I',
 '_id': ObjectId('628ac5d0f8b13d75e14120d7'),
 'air_temperature_celcius': 24,
 'date': '2021-01-04',
 'hotspots': [],
 'max_wind_sp

In [4]:
# Find the latest date from the model
Date = Embedded.aggregate([
    { "$sort" : {"date":-1}},
    { "$project" : {"date":1}},
    { "$limit": 1 }
])
for data in Date:
    pprint(data)   

{'_id': ObjectId('628ac5d0f8b13d75e1412240'), 'date': '2021-12-31'}


Here we can see that 2021-12-31 is the latest date that the climate data was inserted into mongo from Part A data model

In [5]:
# import statements
from time import sleep
from json import dumps
from kafka import KafkaProducer
import random
import datetime as dt
import csv

In [6]:
def publish_message(producer_instance, topic_name, data):
    try:
        value_bytes = bytes(data, encoding='utf-8')
        producer_instance.send(topic_name, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully. Data: ' + data)
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

In [7]:
# Establish Connection with Kafka
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    

In [8]:
if __name__ == '__main__':
   
    topic = 'EventProducer01'
    
    print('Publishing records..')
    producer = connect_kafka_producer()
    
    # Use the latest date from above
    latest_date = dt.date(2021,12,31)
    new_date = latest_date + dt.timedelta(days=1)
    
    # Open the csv file
    with open('climate_streaming.csv') as climate_streaming:
        climate = csv.DictReader(climate_streaming)
        optionC = [i for i in climate]
        # Run for 100 times
        for e in range(100):
            # random choice
            chosen_row = random.choice(optionC)
            # assign date and producer number to the data
            chosen_row['date'] = new_date.strftime("%Y-%m-%d")
            chosen_row['producer_num'] = 'Producer01'
            data = str(chosen_row)
            # push data into kafka
            publish_message(producer, topic, data)
            # wait for 10 seconds before going again
            sleep(10)
            new_date = new_date + dt.timedelta(days=1)

Publishing records..
Message published successfully. Data: {'latitude': '-37.758', 'longitude': '144.693', 'air_temperature_celcius': '20', 'relative_humidity': '58.8', 'windspeed_knots': '11.5', 'max_wind_speed': '15.9', 'precipitation ': ' 0.00I', 'GHI_w/m2': '159', 'date': '2022-01-01', 'producer_num': 'Producer01'}
Message published successfully. Data: {'latitude': '-34.5374', 'longitude': '141.7141', 'air_temperature_celcius': '8', 'relative_humidity': '40.8', 'windspeed_knots': '5.5', 'max_wind_speed': '9.9', 'precipitation ': ' 0.01G', 'GHI_w/m2': '74', 'date': '2022-01-02', 'producer_num': 'Producer01'}
Message published successfully. Data: {'latitude': '-37.45', 'longitude': '148.097', 'air_temperature_celcius': '9', 'relative_humidity': '41', 'windspeed_knots': '8.2', 'max_wind_speed': '9.9', 'precipitation ': ' 0.00G', 'GHI_w/m2': '83', 'date': '2022-01-03', 'producer_num': 'Producer01'}
Message published successfully. Data: {'latitude': '-36.4025', 'longitude': '142.5598', 